<img align='left' alt='ESO Logo' src='http://archive.eso.org/i/esologo.png'>  

# &nbsp; ESO Science Archive Programmatic: HOWTOs

##  HOW TO obtain archival ESO data given RA and Dec and investigate using Aladin

This script follows the inital ``HOW TO obtain archival ESO data given RA and Dec`` script. 

However, with this example, we show how to obtain some images towards the Galacitc Centre from the [GALACTICNUCLEUS](https://www.aanda.org/articles/aa/full_html/2019/11/aa36263-19/aa36263-19.html) project (HAWKI) and veiw these in [Aladin Sky Atlas](https://aladin.cds.unistra.fr/). 

In case you use data from the ESO Archive for your research, please consider the [ESO acknowledgement policies](http://archive.eso.org/cms/eso-data-access-policy.html#acknowledgement).

**Usage**: You can access this file as an interactive jupyter notebook, which you can download and run on your machine [(instructions)](https://jupyter.org/install). To interact with the jupyter notebook (if you have download it): move up and down the various cells using the arrow keys, execute the code by pressing CTRL+ENTER; you can also modify the code and execute it at will.

**Version**: Please note that this notebook uses Python version 3.10.15 and Jupyter Notebook version 6.4.10. Compatibility with other versions cannot be guaranteed.

**Authors**: Ashley Thomas Barnes

---
## <a id='imports'>0. Imports - inc. ESOAsg package</a>

In [1]:
from ESOAsg import archive_observations # ESO Archive Observations
from ESOAsg import archive_science_portal # ESO Archive Science Portal

from astropy.coordinates import SkyCoord # Astropy coordinates
from astropy.io import fits # Astropy FITS files
import astropy.constants as const # Astropy constants
import astropy.units as u # Astropy units
import matplotlib.pyplot as plt # Matplotlib
import numpy as np # Numpy
from PyAstronomy import pyasl # PyAstronomy - used for Baricentric correction if not in header

# Set the default font size and weight for plots
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['xtick.top'] = True
plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['ytick.right'] = True
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['figure.dpi'] = 200

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


---
## <a id='RA/Dec'>1. Get RA and Dec of one target (or more) </a>

To begin, you need to choose the targets you would like to explore and get their coordinates. These need to be an [astropy.coordinates](https://docs.astropy.org/en/stable/coordinates/) `SkyCoord` object. A `SkyCoord` object can easily be created by using:
```python
from astropy import units as u
from astropy.coordinates import SkyCoord
coordinates = SkyCoord(ra=266.41681662*u.degree, dec=-29.00782497*u.degree, frame='icrs')
```
For this example, we consider an example for ``Sgr A*`` - the Supermassive blackhole at the centre of the Milky Way.

In [2]:
target_name = 'SgrA*' 
target_position = SkyCoord.from_name(target_name)
print('{}: {}'.format(target_name, target_position.to_string('hmsdms')))

SgrA*: 17h45m40.0359888s -29d00m28.169892s


---
## <a id='query'>2. Query the ESO archive and download data </a>

The function <code>archive_observations.query_from_radec</code> allows to explore the processed data (also known as ADPs) within a certain radius in arcseconds (e.g., radius=5.) from a given position. 

It is also possible to limit the search to specific instruments (e.g., ``HAWKI``) or specific data types (e.g., ``image``).

In [3]:
# Query the ESO archive for observations of the stars
result_from_queries = archive_observations.query_from_radec(target_position, # Target position
                                                            radius=5, # Radius in arcsec
                                                            data_types='image', # Data type
                                                            instruments=['HAWKI'] # Instrument
                                                            ) 

[WORKING] :: Running query 1 to the ESO archive (out of 1 total)
[INFO]    :: A total of 24 entries has been retrieved


Here we filter our results for the collection ``195.B-0283`` which is from the [GALACTICNUCLEUS](https://www.aanda.org/articles/aa/full_html/2019/11/aa36263-19/aa36263-19.html) survey. 

In [4]:
result_from_queries = result_from_queries[result_from_queries['obs_collection'] == '195.B-0283'] # Select the HAWKI observations from the GalacticNucleus data release
result_from_queries.show_in_notebook(show_row_index=False) # Show the table in a Jupyter notebook

         interactive tables it is recommended to use dedicated tools like:
         - https://github.com/bloomberg/ipydatagrid
         - https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html#datatable
         - https://dash.plotly.com/datatable [warnings]


target_name,dp_id,s_ra,s_dec,t_exptime,em_min,em_max,dataproduct_type,instrument_name,obstech,abmaglim,proposal_id,obs_collection
,,deg,deg,s,m,m,,,,mag,,
FAST-SGRA-1,ADP.2021-04-30T12:05:58.659,266.41302,-29.042113,1175.015,1.181e-06,1.335e-06,image,HAWKI,"IMAGE,HIT",22.0,195.B-0283(A),195.B-0283
FAST-SGRA-1,ADP.2021-04-30T12:05:58.891,266.41302,-29.042113,1175.015,1.474e-06,1.764e-06,image,HAWKI,"IMAGE,HIT",21.0,195.B-0283(A),195.B-0283
FAST-SGRA-1,ADP.2021-04-30T12:05:58.893,266.38847,-29.029074,1175.015,1.474e-06,1.764e-06,image,HAWKI,"IMAGE,HIT",21.0,195.B-0283(A),195.B-0283
FAST-SGRA-1,ADP.2021-04-30T12:05:58.663,266.427,-28.973507,1175.015,1.181e-06,1.335e-06,image,HAWKI,"IMAGE,HIT",22.0,195.B-0283(A),195.B-0283
FAST-SGRA-1,ADP.2021-04-30T12:05:58.895,266.427,-28.973507,1175.015,1.474e-06,1.764e-06,image,HAWKI,"IMAGE,HIT",21.0,195.B-0283(A),195.B-0283
FAST-SGRA-1,ADP.2021-04-30T12:05:58.657,266.45155,-28.986542,1175.015,1.181e-06,1.335e-06,image,HAWKI,"IMAGE,HIT",22.0,195.B-0283(A),195.B-0283
FAST-SGRA-1,ADP.2021-04-30T12:05:58.889,266.45155,-28.986541,1175.015,1.474e-06,1.764e-06,image,HAWKI,"IMAGE,HIT",21.0,195.B-0283(A),195.B-0283
FAST-SGRA-1,ADP.2021-04-30T12:05:58.661,266.38847,-29.029075,1175.015,1.181e-06,1.335e-06,image,HAWKI,"IMAGE,HIT",22.0,195.B-0283(A),195.B-0283
FAST-SGRA-1,ADP.2021-04-30T12:05:59.255,266.41302,-29.042114,1175.015,1.982e-06,2.307e-06,image,HAWKI,"IMAGE,HIT",21.0,195.B-0283(A),195.B-0283


### <a id='aladin_preview'>2.1 Aladin Preview </a>

We open this in an ``Aladin Lite`` preview using the below command. 

```python
import archive_science_portal
archive_science_portal.check_hips()
```

**Note:** These will open links though the browser, and if providing multiple dp_ids this can be a number of tabs.

In [5]:
# Get the data product ID of the first observation
dp_id = result_from_queries['dp_id'][0]

# This opens the HIPS in Aladin Lite preview 
archive_science_portal.check_hips(dp_id)

[INFO]    :: This will open a new tab with the HIPS information in browser


We now download the data the usual way... 

In [6]:
# Specify the parent directory; update this to match your own directory structure
output_dir = './data/' 

# Download the data products
output_files = archive_observations.download(dp_id, output_dir=output_dir)

[INFO]    :: Your disk has:
[INFO]    :: Total: 926.35 GB, Used: 683.35 GB, Free: 243.00 GB
[WORKING] :: Retrieving file ADP.2021-04-30T12:05:58.659.fits
[INFO]    :: File ADP.2021-04-30T12:05:58.659.fits downloaded


---
## <a id='aladin'>2. View is Aladin Lite </a>

We now make use of the [ipyaladin](https://cds-astro.github.io/ipyaladin/getting_started/getting_started.html) package to view the data in [Aladin Lite](https://aladin.cds.unistra.fr/AladinLite/). 

The package should be easy to install through.

```bash
pip install ipyaladin
```


In [7]:
from ipyaladin import Aladin # Aladin Lite widget for Jupyter notebooks

We start by creating an Aladin Lite widget and setting the target position as the initial view. 

Here we show the [PanSTARRS](https://outerspace.stsci.edu/display/PANSTARRS/) survey data towards the central 0.2 degrees of the Galactic Centre (in galactic coordinates).

In [8]:
aladin = Aladin(fov=0.2, # Field of view in degrees
                survey='P/PanSTARRS/DR1/color-i-r-g', # Survey to display
                target=target_name, # Target to display
                coo_frame="galactic") # Coordinate frame to use
aladin

Aladin(coo_frame='galactic', survey='P/PanSTARRS/DR1/color-i-r-g')

We can now load the image we downloaded from the ESO Archive into Aladin Lite widget.

Note that the image is displayed in the same frame as the Aladin Lite widget (above).

In [9]:
hdu = fits.open(f'{output_dir}{output_files[0]}')[0] # Open the FITS file downloaded from the ESO archive
aladin.add_fits(hdu, name=target_name, opacity=0.75) # Add the FITS file to the Aladin Lite widget

We can now save this as an image to view later.

In [12]:
aladin.save_view_as_image("./figures/aladin_GALACTICNUCLEUS.png")